In [1]:
import pandas as pd
import numpy as np
import pickle
import logging
import json

from model_metrics import format_results
import data_clean_for_model
import PipelineHelper
from sklearn.preprocessing import StandardScaler

/Users/benji/anaconda3/envs/play/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
logging.basicConfig(level = logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
### 1. Load Data
df = pd.read_parquet("data/all_processed_df.parquet.gzip")

In [4]:
k = 5
rseed = 229
df["outcome"] = np.where( df["state"]=="successful", 1, 0, )
df["un_id"] = np.arange(0, df.shape[0], 1 )
df["name_len"] = df["name"].str.len()
df["cv_group"] = np.random.choice( np.arange(0, k), size=df.shape[0] )
df["binned_usd_goal"] = pd.qcut( np.log(df["usd_goal"]+1), 20 )

with open("model_config.json", 'r') as j:
     model_params = json.loads(j.read())
model_params['naive_bayes']['ngram_range'] = tuple(model_params['naive_bayes']['ngram_range'])

In [5]:
## load project metadata
logger.info("Loading features")
try:
    f = open("data/features.pkl", "rb")
    ft_dict = pickle.load(f)
    f.close()
    X_train, y_train, X_test, y_test = ft_dict.values()
except:
    X_train, X_test, y_train, y_test = data_clean_for_model.data_clean_for_model(df, "outcome", model_params, cv=model_params["cv"])

INFO:__main__:Loading features


In [6]:
# load text
logger.info("Processing text data")
blurb_train, blurb_test, _, _    = data_clean_for_model.process_blurb(df, model_params)

INFO:__main__:Processing text data


In [7]:
## 2. Run text models

try: 
    f = open("data/res/text_models.pkl", "rb")
    text_models = pickle.load(f)
    f.close()
except:
    raise Warning("Text models do not exist. Will load from scratch")
# get naive bayes predictions
logger.info("Loading Naive Bayes predictions")
try:
    #nb_proba_train = np.load("data/res/multi_nb_preds_train.npy")
    #nb_proba_test = np.load("data/res/multi_nb_preds_test.npy")
    nb_proba_train, nb_proba_test = text_models['nb_train'], text_models['nb_test']
except:
    logger.info("Running Naive Bayes model")
    nb_params = model_params['naive_bayes']
    nb_train_pred, nb_proba_train, nb_test_pred, nb_proba_test = PipelineHelper.naive_bayes_predictions(
        blurb_train, y_train, blurb_test,
        tfidf=nb_params['tf-idf'], ngram_range=nb_params['ngram_range']
    )
    np.save("data/res/multi_nb_preds_train.npy", nb_proba_train)
    np.save("data/res/multi_nb_preds_test.npy", nb_proba_test)

# get LDA topic model
logger.info("Loading LDA topic predictions")
try:
    lda_train, lda_test = text_models['lda_train'], text_models['lda_test']
    #lda_train = pd.read_csv("data/res/lda_train.csv").drop(columns=['Unnamed: 0'])
    #lda_test = pd.read_csv("data/res/lda_test.csv").drop(columns=['Unnamed: 0'])
except:
    logger.info("Running LDA topic model")
    lda_params = model_params['lda']
    tokenized_train = blurb_train.apply(data_clean_for_model.tokenize_text)
    tokenized_test = blurb_test.apply(data_clean_for_model.tokenize_text)
    lda_train, lda_test = PipelineHelper.train_lda_model(tokenized_train, tokenized_test, params['lda'])
    lda_train.to_csv("data/res/lda_train.csv")
    lda_test.to_csv("data/res/lda_test.csv")

# get Word2Vec model predictions
logger.info("Loading Word2Vec dimension predictions")
try:
    #f = open("data/res/w2v_dict.pkl", "rb")
    #w2v_dict = pickle.load(f)
    #f.close()
    #w2v_train, w2v_test = w2v_dict.values()
    w2v_train, w2v_test = text_models['w2v_train'], text_models['w2v_test']
except:
    raise Warning("Word2Vec function not implemented. Running without it -- likely will crash.")

INFO:__main__:Loading Naive Bayes predictions
INFO:__main__:Loading LDA topic predictions
INFO:__main__:Loading Word2Vec dimension predictions


In [8]:
## HACK: change train size (only downside here is that we are not also updating our NB/LDA/W2V performance)
new_train_ind = int(X_test.shape[0]/3)
X_train = pd.concat((X_train, X_test[0:new_train_ind])).reset_index(drop=True)
X_test = X_test[new_train_ind:].reset_index(drop=True)

nb_proba_train = np.vstack((nb_proba_train, nb_proba_test[0:new_train_ind, :]))
nb_proba_test = nb_proba_test[new_train_ind:, :]

lda_train = pd.concat((lda_train, lda_test[0:new_train_ind])).reset_index(drop=True)
lda_test = lda_test[new_train_ind:].reset_index(drop=True)

w2v_train = pd.concat((pd.DataFrame(w2v_train), pd.DataFrame(w2v_test)[0:new_train_ind] )).reset_index(drop=True)
w2v_test = pd.DataFrame(w2v_test)[new_train_ind:].reset_index(drop=True)

y_train = pd.concat((y_train, y_test[0:new_train_ind])).reset_index(drop=True)
y_test = y_test[new_train_ind:].reset_index(drop=True)

assert X_train.shape[0] == nb_proba_train.shape[0] == lda_train.shape[0] == w2v_train.shape[0]
assert X_test.shape[0] == nb_proba_test.shape[0] == lda_test.shape[0] == w2v_test.shape[0]

In [8]:
import tensorflow as tf
from tensorflow import keras

In [10]:
from tensorflow.keras.models import Sequential 
from keras.layers import Dense

In [11]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [12]:
from sklearn.model_selection import StratifiedKFold

In [17]:
id_vars = ["un_id", "cv_group"]
id_train = X_train[id_vars]
id_test = X_test[id_vars]
X_train = X_train.drop(columns=id_vars)
X_test = X_test.drop(columns=id_vars)

In [18]:
X_train_nb = X_train.copy()
X_test_nb = X_test.copy()
# NB 
X_train_nb['nb_proba'] = nb_proba_train[:, 1]
X_test_nb['nb_proba'] = nb_proba_test[:, 1]

In [19]:
scaler = StandardScaler()
time_vars = ["deadline", "launched_at", "time_diff"]
scale_vars = ["blurb_len", "name_len"] + time_vars
X_train_nb_scale = X_train_nb.copy()
X_test_nb_scale = X_test_nb.copy()
X_train_nb_scale[ scale_vars ] = scaler.fit_transform(X_train_nb[ scale_vars ])
X_test_nb_scale[ scale_vars ] = scaler.transform(X_test_nb[ scale_vars ])
X_train_nb_scale2 = X_train_nb_scale.copy()
X_test_nb_scale2 = X_test_nb_scale.copy()
usd_goal_cols = X_train_nb_scale2.columns[X_train_nb_scale2.columns.str.contains("usd_goal")]
X_train_nb_scale2[ usd_goal_cols ] = scaler.fit_transform(X_train_nb_scale2[ usd_goal_cols ])
X_test_nb_scale2[ usd_goal_cols ] = scaler.transform(X_test_nb_scale2[ usd_goal_cols ])

In [31]:
def create_baseline():
    model = Sequential()
    model.add(Dense(3, input_dim=X_train_nb_scale2.shape[1], activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
    return model 

est = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=10, verbose=1)
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=229)
results = cross_val_score(est, X_train_nb_scale2.iloc[0:10000], y_train[0:10000], cv=kfold, verbose=5, n_jobs=4)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   5 | elapsed:  2.1min remaining:  3.2min


Baseline: 83.02% (1.09%)


[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:  3.2min finished


In [33]:
def create_baseline():
    model = Sequential()
    model.add(Dense(5, input_dim=X_train_nb_scale2.shape[1], activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
    return model 
est = KerasClassifier(build_fn=create_baseline, epochs=30, batch_size=5, verbose=1)
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=229)
results = cross_val_score(est, X_train_nb_scale2, y_train, cv=kfold, verbose=5, n_jobs=4)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   5 | elapsed: 24.7min remaining: 37.0min
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed: 36.3min finished


Baseline: 84.82% (0.55%)


In [34]:
results

array([0.85101694, 0.84870058, 0.85384178, 0.8495155 , 0.83782029])

In [34]:
def create_baseline(input_layer=25):
    model = Sequential()
    model.add(Dense(input_layer, input_dim=X_train_nb_scale2.shape[1], activation='selu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
    return model 

In [36]:
mod = create_baseline()
mod.fit(X_train_nb_scale2, y_train, validation_data=(X_test_nb_scale2,y_test), batch_size = 5, epochs = 20, class_weight = {0:1.24764767, 1:0.8343821})
y_pred = mod.predict(X_test_nb_scale2)
y_pred
(np.round(y_pred).flatten()==y_test).sum() / y_test.shape[0]

Epoch 1/20
30975/30975 [==============================] - 45s 1ms/step - loss: 0.3459 - accuracy: 0.8290 - val_loss: 0.3890 - val_accuracy: 0.8078
Epoch 2/20
30975/30975 [==============================] - 39s 1ms/step - loss: 0.3211 - accuracy: 0.8432 - val_loss: 0.3872 - val_accuracy: 0.8050
Epoch 3/20
30975/30975 [==============================] - 44s 1ms/step - loss: 0.3158 - accuracy: 0.8466 - val_loss: 0.3993 - val_accuracy: 0.8040
Epoch 4/20
30975/30975 [==============================] - 43s 1ms/step - loss: 0.3128 - accuracy: 0.8483 - val_loss: 0.3747 - val_accuracy: 0.8114
Epoch 5/20
30975/30975 [==============================] - 43s 1ms/step - loss: 0.3109 - accuracy: 0.8486 - val_loss: 0.3737 - val_accuracy: 0.8131
Epoch 6/20
30975/30975 [==============================] - 49s 2ms/step - loss: 0.3095 - accuracy: 0.8497 - val_loss: 0.3875 - val_accuracy: 0.8102
Epoch 7/20
30975/30975 [==============================] - 45s 1ms/step - loss: 0.3082 - accuracy: 0.8502 - val_loss: 0

0.815683615819209

In [37]:
np.round(calculate_performance(y_test, np.round(y_pred.ravel())),2)

array([0.82, 0.84, 0.88, 0.74, 0.8 , 0.84, 0.82, 0.18])

In [38]:
mod.save("data/nns/metadata_nb_nn_batch5_epochs30_dim_25.tf")

INFO:tensorflow:Assets written to: data/nns/metadata_nb_nn_batch5_epochs30_dim_25.tf/assets


In [28]:
mod = create_baseline(50)
mod.fit(X_train_nb_scale2, y_train, validation_data=(X_test_nb_scale2,y_test), batch_size = 3, epochs = 10)
y_pred = mod.predict(X_test_nb_scale2)
y_pred
(np.round(y_pred).flatten()==y_test).sum() / y_test.shape[0]

Epoch 1/10
51625/51625 [==============================] - 64s 1ms/step - loss: 0.3410 - accuracy: 0.8359 - val_loss: 0.3771 - val_accuracy: 0.8091
Epoch 2/10
51625/51625 [==============================] - 56s 1ms/step - loss: 0.3169 - accuracy: 0.8492 - val_loss: 0.3815 - val_accuracy: 0.8079
Epoch 3/10
51625/51625 [==============================] - 62s 1ms/step - loss: 0.3118 - accuracy: 0.8518 - val_loss: 0.3752 - val_accuracy: 0.8124
Epoch 4/10
51625/51625 [==============================] - 57s 1ms/step - loss: 0.3091 - accuracy: 0.8531 - val_loss: 0.3758 - val_accuracy: 0.8083
Epoch 5/10
51625/51625 [==============================] - 57s 1ms/step - loss: 0.3073 - accuracy: 0.8542 - val_loss: 0.3711 - val_accuracy: 0.8115
Epoch 6/10
51625/51625 [==============================] - 60s 1ms/step - loss: 0.3058 - accuracy: 0.8540 - val_loss: 0.3774 - val_accuracy: 0.8122
Epoch 7/10
51625/51625 [==============================] - 57s 1ms/step - loss: 0.3048 - accuracy: 0.8553 - val_loss: 0

0.8114500941619586

In [ ]:
mod.save("data/nns/nn_batch5_epochs7_dim50.tf")

In [15]:
def create_baseline(input_layer=25):
    model = Sequential()
    model.add(Dense(input_layer, input_dim=X_train_nb_scale2.shape[1], activation='selu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
    return model 
mod = create_baseline(25)
mod.fit(X_train_nb_scale2, y_train, validation_data=(X_test_nb_scale2,y_test), batch_size = 5, epochs = 15)
y_pred = mod.predict(X_test_nb_scale2)
y_pred
(np.round(y_pred).flatten()==y_test).sum() / y_test.shape[0]

Epoch 1/15
13417/30975 [===========>..................] - ETA: 18s - loss: 23.5298 - accuracy: 0.5795 ETA: 19s - loss: 23.8

KeyboardInterrupt: 

In [29]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train),
                                                 y_train.reset_index(drop=True))
class_weights

/Users/benji/anaconda3/envs/play/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass classes=[0 1], y=0         1
1         0
2         1
3         0
4         1
         ..
154868    1
154869    1
154870    1
154871    1
154872    1
Name: outcome, Length: 154873, dtype: int64 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


array([1.24764767, 0.8343821 ])

In [30]:
def create_baseline(input_layer=25):
    model = Sequential()
    model.add(Dense(input_layer, input_dim=X_train_nb_scale2.drop(columns="nb_proba").shape[1], activation='selu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
    return model 
mod = create_baseline(25)
mod.fit(X_train_nb_scale2.drop(columns="nb_proba"), y_train, validation_data=(X_test_nb_scale2.drop(columns="nb_proba"),y_test), batch_size = 5, epochs = 15, 
       class_weight = {0:1.24764767, 1:0.8343821}
       )
y_pred = mod.predict(X_test_nb_scale2.drop(columns="nb_proba"))
y_pred
(np.round(y_pred).flatten()==y_test).sum() / y_test.shape[0]

Epoch 1/15
30975/30975 [==============================] - 38s 1ms/step - loss: 0.4089 - accuracy: 0.7789 - val_loss: 0.3958 - val_accuracy: 0.7915
Epoch 2/15
30975/30975 [==============================] - 39s 1ms/step - loss: 0.3859 - accuracy: 0.7918 - val_loss: 0.3841 - val_accuracy: 0.7973
Epoch 3/15
30975/30975 [==============================] - 38s 1ms/step - loss: 0.3789 - accuracy: 0.7955 - val_loss: 0.3871 - val_accuracy: 0.7966
Epoch 4/15
30975/30975 [==============================] - 39s 1ms/step - loss: 0.3745 - accuracy: 0.7975 - val_loss: 0.3907 - val_accuracy: 0.7900
Epoch 5/15
30975/30975 [==============================] - 38s 1ms/step - loss: 0.3724 - accuracy: 0.7983 - val_loss: 0.3782 - val_accuracy: 0.7988
Epoch 6/15
30975/30975 [==============================] - 40s 1ms/step - loss: 0.3705 - accuracy: 0.7992 - val_loss: 0.3851 - val_accuracy: 0.7943
Epoch 7/15
30975/30975 [==============================] - 38s 1ms/step - loss: 0.3693 - accuracy: 0.7992 - val_loss: 0

0.799984934086629

In [33]:
mod.save("data/nns/metadata_nn_batch5_epoch15_dim25.tf")

INFO:tensorflow:Assets written to: data/nns/metadata_nn_batch5_epoch15_dim25.tf/assets


In [31]:
from model_metrics import calculate_performance

In [32]:
np.round(calculate_performance(y_test, np.round(y_pred.ravel())),2)

array([0.8 , 0.82, 0.9 , 0.7 , 0.75, 0.88, 0.81, 0.2 ])

INFO:tensorflow:Assets written to: data/nns/nn_batch5_epochs7_dim50.tf/assets


In [30]:
X_train_nb_lda_scale = pd.concat((X_train_nb_scale2, lda_train), axis=1).copy() 
X_test_nb_lda_scale =pd.concat((X_test_nb_scale2, lda_test), axis=1).copy() 
X_train_nb_lda_scale[ lda_train.columns ] = scaler.fit_transform(X_train_nb_lda_scale[ lda_train.columns ])
X_test_nb_lda_scale[ lda_train.columns ] = scaler.transform(X_test_nb_lda_scale[ lda_train.columns ])

In [37]:
def create_baseline(input_layer=25):
    model = Sequential()
    model.add(Dense(input_layer, input_dim=X_train_nb_lda_scale.shape[1], activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
    return model 
mod = create_baseline(50)
mod.fit(X_train_nb_lda_scale, y_train, validation_data=(X_test_nb_lda_scale,y_test), batch_size = 5, epochs = 7)
y_pred = mod.predict(X_test_nb_lda_scale)
y_pred
(np.round(y_pred).flatten()==y_test).sum() / y_test.shape[0]

Epoch 1/7
30975/30975 [==============================] - 44s 1ms/step - loss: 0.3411 - accuracy: 0.8342 - val_loss: 0.3798 - val_accuracy: 0.8090
Epoch 2/7
30975/30975 [==============================] - 37s 1ms/step - loss: 0.3126 - accuracy: 0.8503 - val_loss: 0.3701 - val_accuracy: 0.8124
Epoch 3/7
30975/30975 [==============================] - 78s 3ms/step - loss: 0.3051 - accuracy: 0.8550 - val_loss: 0.3763 - val_accuracy: 0.8112
Epoch 4/7
30975/30975 [==============================] - 66s 2ms/step - loss: 0.3002 - accuracy: 0.8572 - val_loss: 0.3681 - val_accuracy: 0.8152
Epoch 5/7
30975/30975 [==============================] - 53s 2ms/step - loss: 0.2973 - accuracy: 0.8596 - val_loss: 0.3697 - val_accuracy: 0.8133
Epoch 6/7
30975/30975 [==============================] - 44s 1ms/step - loss: 0.2949 - accuracy: 0.8610 - val_loss: 0.3686 - val_accuracy: 0.8149
Epoch 7/7
30975/30975 [==============================] - 46s 1ms/step - loss: 0.2931 - accuracy: 0.8620 - val_loss: 0.3772 -

0.8121431261770244

In [33]:
mod2 = create_baseline(25)
mod2.fit(X_train_nb_lda_scale, y_train, validation_data=(X_test_nb_lda_scale,y_test), batch_size = 5, epochs = 7)
y_pred = mod2.predict(X_test_nb_lda_scale)
y_pred
(np.round(y_pred).flatten()==y_test).sum() / y_test.shape[0]

Epoch 1/7
30975/30975 [==============================] - 38s 1ms/step - loss: 0.3490 - accuracy: 0.8312 - val_loss: 0.3885 - val_accuracy: 0.8044
Epoch 2/7
30975/30975 [==============================] - 35s 1ms/step - loss: 0.3201 - accuracy: 0.8474 - val_loss: 0.3864 - val_accuracy: 0.8059
Epoch 3/7
30975/30975 [==============================] - 35s 1ms/step - loss: 0.3126 - accuracy: 0.8509 - val_loss: 0.3764 - val_accuracy: 0.8079
Epoch 4/7
30975/30975 [==============================] - 33s 1ms/step - loss: 0.3086 - accuracy: 0.8539 - val_loss: 0.3705 - val_accuracy: 0.8126
Epoch 5/7
30975/30975 [==============================] - 33s 1ms/step - loss: 0.3058 - accuracy: 0.8546 - val_loss: 0.3665 - val_accuracy: 0.8138
Epoch 6/7
30975/30975 [==============================] - 33s 1ms/step - loss: 0.3041 - accuracy: 0.8553 - val_loss: 0.3701 - val_accuracy: 0.8121
Epoch 7/7
30975/30975 [==============================] - 35s 1ms/step - loss: 0.3023 - accuracy: 0.8571 - val_loss: 0.3660 -

0.8138154425612053

In [34]:
X_train_nb_w2v_scale = pd.concat((X_train_nb_scale2, w2v_train), axis=1).copy() 
X_test_nb_w2v_scale =pd.concat((X_test_nb_scale2, w2v_test), axis=1).copy() 
X_train_nb_w2v_scale[ w2v_train.columns ] = scaler.fit_transform(X_train_nb_w2v_scale[ w2v_train.columns ])
X_test_nb_w2v_scale[ w2v_train.columns ] = scaler.transform(X_test_nb_w2v_scale[ w2v_train.columns ])

In [38]:
def create_baseline(input_layer=25):
    model = Sequential()
    model.add(Dense(input_layer, input_dim=X_train_nb_w2v_scale.shape[1], activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
    return model 
mod3 = create_baseline(25)
mod3.fit(X_train_nb_w2v_scale, y_train, validation_data=(X_test_nb_w2v_scale,y_test), batch_size = 5, epochs = 10)
y_pred = mod3.predict(X_test_nb_w2v_scale)
y_pred
(np.round(y_pred).flatten()==y_test).sum() / y_test.shape[0]

Epoch 1/10
30975/30975 [==============================] - 49s 2ms/step - loss: 0.3863 - accuracy: 0.8079 - val_loss: 0.3821 - val_accuracy: 0.8070
Epoch 2/10
30975/30975 [==============================] - 40s 1ms/step - loss: 0.3267 - accuracy: 0.8425 - val_loss: 0.3796 - val_accuracy: 0.8087
Epoch 3/10
30975/30975 [==============================] - 41s 1ms/step - loss: 0.3063 - accuracy: 0.8547 - val_loss: 0.3841 - val_accuracy: 0.8103
Epoch 4/10
30975/30975 [==============================] - 39s 1ms/step - loss: 0.2957 - accuracy: 0.8616 - val_loss: 0.4027 - val_accuracy: 0.8022
Epoch 5/10
30975/30975 [==============================] - 39s 1ms/step - loss: 0.2876 - accuracy: 0.8663 - val_loss: 0.3859 - val_accuracy: 0.8080
Epoch 6/10
30975/30975 [==============================] - 39s 1ms/step - loss: 0.2819 - accuracy: 0.8694 - val_loss: 0.3937 - val_accuracy: 0.8065
Epoch 7/10
30975/30975 [==============================] - 35s 1ms/step - loss: 0.2769 - accuracy: 0.8716 - val_loss: 0

0.8066290018832392

In [39]:
def create_baseline(input_layer=25):
    model = Sequential()
    model.add(Dense(input_layer, input_dim=tmp_train.shape[1], activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
    return model 
tmp_train = scaler.fit_transform(X_train_nb)
tmp_test = scaler.transform(X_test_nb)
tmp_mod = create_baseline(25)
tmp_mod.fit(tmp_train, y_train, validation_data=(tmp_test,y_test), batch_size = 5, epochs = 10)
y_pred = tmp_mod.predict(tmp_test)
y_pred
(np.round(y_pred).flatten()==y_test).sum() / y_test.shape[0]

Epoch 1/10
30975/30975 [==============================] - 36s 1ms/step - loss: 0.3291 - accuracy: 0.8435 - val_loss: 0.3744 - val_accuracy: 0.8120
Epoch 2/10
30975/30975 [==============================] - 34s 1ms/step - loss: 0.3087 - accuracy: 0.8543 - val_loss: 0.3735 - val_accuracy: 0.8136
Epoch 3/10
30975/30975 [==============================] - 36s 1ms/step - loss: 0.3044 - accuracy: 0.8559 - val_loss: 0.3780 - val_accuracy: 0.8137
Epoch 4/10
30975/30975 [==============================] - 40s 1ms/step - loss: 0.3017 - accuracy: 0.8579 - val_loss: 0.3716 - val_accuracy: 0.8155
Epoch 5/10
30975/30975 [==============================] - 34s 1ms/step - loss: 0.3006 - accuracy: 0.8586 - val_loss: 0.3689 - val_accuracy: 0.8171
Epoch 6/10
30975/30975 [==============================] - 34s 1ms/step - loss: 0.2996 - accuracy: 0.8589 - val_loss: 0.3666 - val_accuracy: 0.8147
Epoch 7/10
30975/30975 [==============================] - 33s 1ms/step - loss: 0.2990 - accuracy: 0.8593 - val_loss: 0

0.8178229755178907

In [41]:
tmp_mod2 = create_baseline(35)
tmp_mod2.fit(tmp_train, y_train, validation_data=(tmp_test,y_test), batch_size = 5, epochs = 14)
y_pred = tmp_mod2.predict(tmp_test)
y_pred
(np.round(y_pred).flatten()==y_test).sum() / y_test.shape[0]

Epoch 1/14
30975/30975 [==============================] - 42s 1ms/step - loss: 0.3304 - accuracy: 0.8427 - val_loss: 0.3765 - val_accuracy: 0.8123
Epoch 2/14
30975/30975 [==============================] - 34s 1ms/step - loss: 0.3086 - accuracy: 0.8541 - val_loss: 0.3739 - val_accuracy: 0.8126
Epoch 3/14
30975/30975 [==============================] - 44s 1ms/step - loss: 0.3039 - accuracy: 0.8562 - val_loss: 0.3750 - val_accuracy: 0.8123
Epoch 4/14
30975/30975 [==============================] - 38s 1ms/step - loss: 0.3026 - accuracy: 0.8582 - val_loss: 0.3695 - val_accuracy: 0.8146
Epoch 5/14
30975/30975 [==============================] - 36s 1ms/step - loss: 0.3007 - accuracy: 0.8583 - val_loss: 0.3674 - val_accuracy: 0.8166
Epoch 6/14
30975/30975 [==============================] - 38s 1ms/step - loss: 0.3001 - accuracy: 0.8591 - val_loss: 0.3758 - val_accuracy: 0.8142
Epoch 7/14
30975/30975 [==============================] - 34s 1ms/step - loss: 0.2990 - accuracy: 0.8594 - val_loss: 0

0.8167532956685499

In [15]:
from sklearn.model_selection import GridSearchCV

In [130]:
def create_baseline(layer=1, output_dim=3, init_activation='sigmoid', final_activation="sigmoid", optimizer='adam'):
    np.random.seed(229)
    tf.random.set_seed(229)
    model = Sequential()
    if layer==1:
        model.add(Dense(output_dim, input_dim=X_train_nb_scale2.shape[1], activation=init_activation))
    if layer==2: 
        model.add(Dense(output_dim, input_dim=X_train_nb_scale2.shape[1], activation=init_activation))
        model.add(Dense(3, activation='relu'))
    if layer==3: 
        model.add(Dense(output_dim, input_dim=X_train_nb_scale2.shape[1], activation=init_activation))
        model.add(Dense(5, activation='relu'))
        model.add(Dense(2, activation='relu'))
    model.add(Dense(1, activation=final_activation))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy']) 
    return model 

In [131]:
model_grid = KerasClassifier(build_fn=create_baseline, verbose=1)

In [135]:
epochs = [75, 100, 150]
batches = [3]
layers = [1]
output_dims = [30]
activations = ['selu']
fin_activations=['sigmoid', 'tanh']
param_grid = dict(epochs=epochs, batch_size=batches, layer=layers, output_dim=output_dims, init_activation=activations, final_activation=fin_activations)
ggrid = GridSearchCV(estimator=model_grid, param_grid=param_grid, cv=2, verbose=3, n_jobs=6)

In [136]:
ggrid.fit(X_train_nb_scale2, y_train, validation_data=(X_test_nb_scale2,y_test))

Fitting 2 folds for each of 6 candidates, totalling 12 fits
Epoch 1/150
59000/59000 [==============================] - 62s 1ms/step - loss: 0.3485 - accuracy: 0.8309 - val_loss: 0.3854 - val_accuracy: 0.8053
Epoch 2/150
59000/59000 [==============================] - 58s 986us/step - loss: 0.3278 - accuracy: 0.8421 - val_loss: 0.3727 - val_accuracy: 0.8126
Epoch 3/150
59000/59000 [==============================] - 56s 945us/step - loss: 0.3220 - accuracy: 0.8452 - val_loss: 0.3722 - val_accuracy: 0.8153
Epoch 4/150
59000/59000 [==============================] - 54s 920us/step - loss: 0.3184 - accuracy: 0.8467 - val_loss: 0.3714 - val_accuracy: 0.8129
Epoch 5/150
59000/59000 [==============================] - 54s 921us/step - loss: 0.3159 - accuracy: 0.8485 - val_loss: 0.3623 - val_accuracy: 0.8164
Epoch 6/150
59000/59000 [==============================] - 56s 943us/step - loss: 0.3143 - accuracy: 0.8489 - val_loss: 0.3621 - val_accuracy: 0.8184
Epoch 7/150
59000/59000 [=================

GridSearchCV(cv=2,
             estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7fd664fdb220>,
             n_jobs=6,
             param_grid={'batch_size': [3], 'epochs': [75, 100, 150],
                         'final_activation': ['sigmoid', 'tanh'],
                         'init_activation': ['selu'], 'layer': [1],
                         'output_dim': [30]},
             verbose=3)

In [137]:
nn_pred = ggrid.predict(X_test_nb_scale2).flatten()
(np.round(nn_pred) == y_test).sum() / y_test.shape[0]

/Users/benji/anaconda3/envs/play/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


14750/14750 [==============================] - 8s 556us/step


0.8195932203389831

In [142]:
# params --> acc 0.820
{'batch_size': 3,
 'epochs': 150,
 'final_activation': 'sigmoid',
 'init_activation': 'selu',
 'layer': 1,
 'output_dim': 30}
ggrid.best_estimator_.model.save("nn.tf")

INFO:tensorflow:Assets written to: nn.tf/assets


In [149]:
X_train_nb_lda = pd.concat((X_train_nb_scale2, lda_train), axis=1)
X_test_nb_lda = pd.concat((X_test_nb_scale2, lda_test), axis=1)
X_train_nb_lda[ lda_train.columns ] = scaler.fit_transform( X_train_nb_lda[ lda_train.columns ] )
X_test_nb_lda[ lda_train.columns ] = scaler.transform( X_test_nb_lda[ lda_train.columns ])

In [155]:
def create_baseline(layer=1, output_dim=3, init_activation='sigmoid', final_activation="sigmoid", optimizer='adam'):
    np.random.seed(229)
    tf.random.set_seed(229)
    model = Sequential()
    if layer==1:
        model.add(Dense(output_dim, input_dim=X_train_nb_lda.shape[1], activation=init_activation))
    if layer==2: 
        model.add(Dense(output_dim, input_dim=X_train_nb_lda.shape[1], activation=init_activation))
        model.add(Dense(output_dim / 2, activation='relu'))
    if layer==3: 
        model.add(Dense(output_dim, input_dim=X_train_nb_lda.shape[1], activation=init_activation))
        model.add(Dense(5, activation='relu'))
        model.add(Dense(2, activation='relu'))
    model.add(Dense(1, activation=final_activation))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy']) 
    return model 

In [160]:
epochs = [30]
batches = [5]
layers = [1]
output_dims = [25]
activations = ['selu']
fin_activations=['sigmoid']
model_grid = KerasClassifier(build_fn=create_baseline, verbose=1)
param_grid = dict(epochs=epochs, batch_size=batches, layer=layers, output_dim=output_dims, init_activation=activations, final_activation=fin_activations)
ggrid = GridSearchCV(estimator=model_grid, param_grid=param_grid, cv=3, verbose=3, n_jobs=6)

In [161]:
ggrid.fit(X_train_nb_lda, y_train, validation_data=(X_test_nb_lda,y_test))

Fitting 3 folds for each of 8 candidates, totalling 24 fits
Epoch 1/30
35400/35400 [==============================] - 45s 1ms/step - loss: 0.3569 - accuracy: 0.8255 - val_loss: 0.3831 - val_accuracy: 0.8042
Epoch 2/30
35400/35400 [==============================] - 43s 1ms/step - loss: 0.3274 - accuracy: 0.8416 - val_loss: 0.3715 - val_accuracy: 0.8115
Epoch 3/30
35400/35400 [==============================] - 37s 1ms/step - loss: 0.3194 - accuracy: 0.8463 - val_loss: 0.3712 - val_accuracy: 0.8153
Epoch 4/30
35400/35400 [==============================] - 38s 1ms/step - loss: 0.3155 - accuracy: 0.8486 - val_loss: 0.3687 - val_accuracy: 0.8138
Epoch 5/30
35400/35400 [==============================] - 61s 2ms/step - loss: 0.3132 - accuracy: 0.8493 - val_loss: 0.3619 - val_accuracy: 0.8159
Epoch 6/30
35400/35400 [==============================] - 76s 2ms/step - loss: 0.3116 - accuracy: 0.8507 - val_loss: 0.3649 - val_accuracy: 0.8158
Epoch 7/30
35400/35400 [==============================] - 

GridSearchCV(cv=3,
             estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7fd5d277d430>,
             n_jobs=6,
             param_grid={'batch_size': [5], 'epochs': [30],
                         'final_activation': ['sigmoid'],
                         'init_activation': ['selu'], 'layer': [1, 2],
                         'output_dim': [20, 30, 50, 100]},
             verbose=3)

In [164]:
ggrid.best_params_

{'batch_size': 5,
 'epochs': 30,
 'final_activation': 'sigmoid',
 'init_activation': 'selu',
 'layer': 1,
 'output_dim': 20}

In [163]:
nn_pred = ggrid.predict(X_test_nb_lda).flatten()
(np.round(nn_pred) == y_test).sum() / y_test.shape[0]

/Users/benji/anaconda3/envs/play/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


8850/8850 [==============================] - 5s 514us/step


0.8143050847457627

In [165]:
from sklearn.svm import SVC

In [166]:
svm = SVC(random_state=229, kernel='rbf')

In [179]:
svm.fit(X_train_nb_w2v[0:10000], y_train[0:10000])

SVC(random_state=229)

In [180]:
ypred = svm.predict(X_test_nb_w2v[0:10000])
(ypred==y_test[0:10000]).sum() / 10000

0.7595

In [178]:
X_train_nb_w2v = pd.concat((X_train_nb_scale2, pd.DataFrame(w2v_train)), axis=1)
X_test_nb_w2v = pd.concat((X_test_nb_scale2, pd.DataFrame(w2v_test)), axis=1)
X_train_nb_w2v[ w2v_train.columns ] = scaler.fit_transform( X_train_nb_w2v[ w2v_train.columns ] )
X_test_nb_w2v[ w2v_train.columns ] = scaler.transform( X_test_nb_w2v[ w2v_train.columns ])

In [171]:
def create_baseline(layer=1, output_dim=3, init_activation='sigmoid', final_activation="sigmoid", optimizer='adam'):
    np.random.seed(229)
    tf.random.set_seed(229)
    model = Sequential()
    if layer==1:
        model.add(Dense(output_dim, input_dim=X_train_nb_w2v.shape[1], activation=init_activation))
    model.add(Dense(1, activation=final_activation))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy']) 
    return model 

In [181]:
epochs = [100]
batches = [3]
layers = [1]
output_dims = [20, 25, 30, 35]
activations = ['selu']
fin_activations=['sigmoid']
model_grid = KerasClassifier(build_fn=create_baseline, verbose=1)
param_grid = dict(epochs=epochs, batch_size=batches, layer=layers, output_dim=output_dims, init_activation=activations, final_activation=fin_activations)
ggrid = GridSearchCV(estimator=model_grid, param_grid=param_grid, cv=3, verbose=3, n_jobs=6)

In [182]:
ggrid.fit(X_train_nb_w2v, y_train, validation_data=(X_test_nb_w2v,y_test))

Fitting 3 folds for each of 4 candidates, totalling 12 fits


KeyboardInterrupt: 

In [186]:
df.loc_id.nunique()

16742

In [124]:
# model w/ params acc -> 0.821
{'batch_size': 5,
 'epochs': 50,
 'init_activation': 'selu',
 'layer': 1,
 'output_dim': 30}
(np.round(nn_pred) == y_test).sum() / y_test.shape[0]

0.8207909604519774